In [1]:
import json
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from gensim.models import KeyedVectors
import gensim.downloader as api

c:\Users\eikes\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = []

with open("example.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [3]:
print(data)

{'dados': [{'entrada': 'carro azul', 'target': 'automóvel azul', 'previsto': 'veículo azul'}, {'entrada': 'computador potente', 'target': 'PC rápido', 'previsto': 'notebook veloz'}, {'entrada': 'banco de madeira', 'target': 'assento de madeira', 'previsto': 'cadeira de madeira'}]}


# FuzzyWuzzy (Fuzzy String Matching)

🔹 O que é?

FuzzyWuzzy usa a **distância de Levenshtein** para medir a semelhança entre duas cadeias de texto.

🔹 Como funciona?

Conta quantas operações (inserção, remoção ou substituição de caracteres) são necessárias para transformar uma string na outra.
Retorna um valor de 0 a 100, onde 100 significa textos idênticos.

In [4]:
for item in data["dados"]:
    score = fuzz.partial_ratio(item["target"], item["previsto"])
    print(f"Target: {item['target']} | Previsto: {item['previsto']} | Similaridade: {score}%")

Target: automóvel azul | Previsto: veículo azul | Similaridade: 70%
Target: PC rápido | Previsto: notebook veloz | Similaridade: 22%
Target: assento de madeira | Previsto: cadeira de madeira | Similaridade: 74%


# Similaridade de Cosseno (TF-IDF)

🔹 O que é?

Mede o ângulo entre vetores que representam os textos. Quanto menor o ângulo, mais parecidas são as frases.

🔹 Como funciona?

Representa cada frase como um vetor numérico usando TF-IDF.
Calcula o cosseno do ângulo entre os vetores.
Retorna um valor de 0 a 1, onde 1 significa frases idênticas.

In [5]:
targets = [item["target"] for item in data["dados"]]
predictions = [item["previsto"] for item in data["dados"]]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(targets + predictions)

for i in range(len(targets)):
    similarity = cosine_similarity(tfidf_matrix[i], tfidf_matrix[i + len(targets)])[0][0]
    print(f"Target: {targets[i]} | Previsto: {predictions[i]} | Similaridade: {similarity:.2f}")

Target: automóvel azul | Previsto: veículo azul | Similaridade: 0.40
Target: PC rápido | Previsto: notebook veloz | Similaridade: 0.00
Target: assento de madeira | Previsto: cadeira de madeira | Similaridade: 0.57


# Sentence Transformer (SBERT)

🔹 O que é?

SentenceTransformer usa modelos baseados em BERT para transformar frases em vetores numéricos de alta dimensão. Depois, podemos calcular a similaridade de cosseno entre esses vetores para medir o quão parecidas as frases são.

🔹 Como funciona?

Converte frases em vetores (embeddings).
Usa similaridade de cosseno para medir a relação entre elas.
Valores mais próximos de 1 indicam frases semelhantes.

In [6]:
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

for item in data["dados"]:
    embedding1 = model.encode(item["target"], convert_to_tensor=True)
    embedding2 = model.encode(item["previsto"], convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
    
    print(f"Target: {item['target']} | Previsto: {item['previsto']} | Similaridade: {similarity:.2f}")

Target: automóvel azul | Previsto: veículo azul | Similaridade: 0.53
Target: PC rápido | Previsto: notebook veloz | Similaridade: 0.28
Target: assento de madeira | Previsto: cadeira de madeira | Similaridade: 0.73


# Word Mover’s Distance (WMD)

🔹 O que é?

Usa Word Embeddings para calcular a menor "distância" que as palavras de uma frase precisam "caminhar" para se tornarem outra frase.

🔹 Como funciona?

Representa palavras como vetores de um modelo de linguagem (ex: Word2Vec, FastText).
Mede o custo de "mover" palavras da frase A para a frase B.
Quanto menor o valor, mais semelhantes as frases.

In [7]:
word_vectors = api.load("fasttext-wiki-news-subwords-300")

def calculate_wmd(sentence1, sentence2, model):
    sentence1 = sentence1.lower().split()
    sentence2 = sentence2.lower().split()
    return model.wmdistance(sentence1, sentence2)

for item in data["dados"]:
    wmd_score = calculate_wmd(item["target"], item["previsto"], word_vectors)
    print(f"Target: {item['target']} | Previsto: {item['previsto']} | WMD: {wmd_score:.4f}")

Target: automóvel azul | Previsto: veículo azul | WMD: 0.0000
Target: PC rápido | Previsto: notebook veloz | WMD: 1.0569
Target: assento de madeira | Previsto: cadeira de madeira | WMD: 0.3476


In [8]:
#http://143.107.183.175:22980/download.php?file=embeddings/fasttext/cbow_s300.zip

modelo_nilc = "../word_embeddings/cbow_s300.txt"
word_vectors = KeyedVectors.load_word2vec_format(modelo_nilc, encoding="utf-8", unicode_errors="ignore")

def calculate_wmd(sentence1, sentence2, model):
    sentence1 = sentence1.lower().split()
    sentence2 = sentence2.lower().split()
    return model.wmdistance(sentence1, sentence2)

for item in data["dados"]:
    wmd_score = calculate_wmd(item["target"], item["previsto"], word_vectors)
    print(f"Target: {item['target']} | Previsto: {item['previsto']} | WMD: {wmd_score:.4f}")

Target: automóvel azul | Previsto: veículo azul | WMD: 0.3304
Target: PC rápido | Previsto: notebook veloz | WMD: 0.9328
Target: assento de madeira | Previsto: cadeira de madeira | WMD: 0.2947


🔹 Se você quer comparar textos curtos, use FuzzyWuzzy ou Levenshtein.

🔹 Se quiser entender o contexto, WMD e Word Embeddings são as melhores opções.

🔹 Para analisar grandes textos, TF-IDF + Similaridade de Cosseno é eficiente.

In [ ]:
model_sbert = SentenceTransformer("paraphrase-MiniLM-L6-v2")
word_vectors_ft = api.load("fasttext-wiki-news-subwords-300")

modelo_nilc = "../word_embeddings/cbow_s300.txt"
word_vectors_nilc = KeyedVectors.load_word2vec_format(modelo_nilc, encoding="utf-8", unicode_errors="ignore")

def normalize_wmd(scores):
    min_score = min(scores)
    max_score = max(scores)
    return [(1 - ((s - min_score) / (max_score - min_score))) if max_score != min_score else 1 for s in scores]

def calculate_wmd(sentence1, sentence2, model):
    sentence1 = sentence1.lower().split()
    sentence2 = sentence2.lower().split()
    return model.wmdistance(sentence1, sentence2)

data = {
    "dados": [
        {"target": "automóvel azul", "previsto": "veículo azul"},
        {"target": "automóvel vermelho", "previsto": "veículo vermelho"},
        {"target": "PC rápido", "previsto": "notebook veloz"},
        {"target": "assento de madeira", "previsto": "cadeira de madeira"},
        {"target": "gato preto", "previsto": "felino escuro"},
        {"target": "avião grande", "previsto": "aeronave gigante"},
        {"target": "nome de pessoa", "previsto": "idade"},
    ]
}

wmd_scores_ft = []
wmd_scores_nilc = []

for item in data["dados"]:
    wmd_scores_ft.append(calculate_wmd(item["target"], item["previsto"], word_vectors_ft))
    wmd_scores_nilc.append(calculate_wmd(item["target"], item["previsto"], word_vectors_nilc))

wmd_scores_ft = normalize_wmd(wmd_scores_ft)
wmd_scores_nilc = normalize_wmd(wmd_scores_nilc)

targets = [item["target"] for item in data["dados"]]
predictions = [item["previsto"] for item in data["dados"]]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(targets + predictions)

for i, item in enumerate(data["dados"]):
    target = item["target"]
    predicted = item["previsto"]

    fuzzy_score = fuzz.partial_ratio(target, predicted) / 100  

    cosine_sim = cosine_similarity(tfidf_matrix[i], tfidf_matrix[i + len(targets)])[0][0]

    embedding1 = model_sbert.encode(target, convert_to_tensor=True)
    embedding2 = model_sbert.encode(predicted, convert_to_tensor=True)
    sbert_score = util.pytorch_cos_sim(embedding1, embedding2).item()

    wmd_score_ft = wmd_scores_ft[i]
    wmd_score_nilc = wmd_scores_nilc[i]

    print(f"Target: {target} | Previsto: {predicted}")
    print(f"  🔹 FuzzyWuzzy: {fuzzy_score:.4f}")
    print(f"  🔹 Similaridade Cosseno (TF-IDF): {cosine_sim:.4f}")
    print(f"  🔹 Similaridade SBERT: {sbert_score:.4f}")
    print(f"  🔹 WMD (FastText - Normalizado): {wmd_score_ft:.4f}")
    print(f"  🔹 WMD (NILC - Normalizado): {wmd_score_nilc:.4f}")
    print("-" * 60)

Target: automóvel azul | Previsto: veículo azul
  🔹 FuzzyWuzzy: 0.7000
  🔹 Similaridade Cosseno (TF-IDF): 0.5000
  🔹 Similaridade SBERT: 0.5279
  🔹 WMD (FastText - Normalizado): 1.0000
  🔹 WMD (NILC - Normalizado): 0.9605
------------------------------------------------------------
Target: automóvel vermelho | Previsto: veículo vermelho
  🔹 FuzzyWuzzy: 0.7700
  🔹 Similaridade Cosseno (TF-IDF): 0.5000
  🔹 Similaridade SBERT: 0.5789
  🔹 WMD (FastText - Normalizado): 1.0000
  🔹 WMD (NILC - Normalizado): 0.9605
------------------------------------------------------------
Target: PC rápido | Previsto: notebook veloz
  🔹 FuzzyWuzzy: 0.2200
  🔹 Similaridade Cosseno (TF-IDF): 0.0000
  🔹 Similaridade SBERT: 0.2839
  🔹 WMD (FastText - Normalizado): 0.0000
  🔹 WMD (NILC - Normalizado): 0.2957
------------------------------------------------------------
Target: assento de madeira | Previsto: cadeira de madeira
  🔹 FuzzyWuzzy: 0.7400
  🔹 Similaridade Cosseno (TF-IDF): 0.5730
  🔹 Similaridade SBERT: